In [1]:
%load_ext autotime
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

In [2]:
import py_entitymatching as em

time: 1.38 s


In [3]:
A = em.load_dataset('person_table_A', key='ID')
B = em.load_dataset('person_table_B', key='ID')

time: 7.83 ms


In [4]:
A

,ID,name,birth_year,hourly_wage,address,zipcode
0,a1,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107
1,a2,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
2,a3,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
3,a4,Binto George,1987,32.5,"423 Powell St, San Francisco",94122
4,a5,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122


time: 31.7 ms


In [5]:
B

,ID,name,birth_year,hourly_wage,address,zipcode
0,b1,Mark Levene,1987,29.5,"108 Clement St, San Francisco",94107
1,b2,Bill Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
2,b3,Mike Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
3,b4,Joseph Kuan,1982,26.0,"108 South Park, San Francisco",94122
4,b5,Alfons Kemper,1984,35.0,"170 Post St, Apt 4, San Francisco",94122
5,b6,Michael Brodie,1987,32.5,"133 Clement Street, San Francisco",94107


time: 17.4 ms


In [6]:
L = em.read_csv_metadata('person_labeled_data.csv', key='_id')

time: 4.34 ms


In [7]:
from py_entitymatching.utils.generic_helper import list_diff

time: 1.25 ms


In [8]:
em.get_key(A)

'ID'

time: 3.94 ms


In [9]:
A1 = A[list_diff(list(A.columns), [em.get_key(A)])]
B1 = B[list_diff(list(B.columns), [em.get_key(B)])]

time: 5.29 ms


In [10]:
feature_table = em.get_features_for_matching(A1, B1)

time: 23.7 ms


In [11]:
em.set_fk_ltable(L, 'ltable_ID')
em.set_fk_rtable(L, 'rtable_ID')
em.set_ltable(L, A)
em.set_rtable(L, B)

True

time: 9.42 ms


In [12]:
E = em.extract_feature_vecs(L, feature_table=feature_table, attrs_after='label')

0%      100%
[##########] | ETA: 00:00:00

time: 92.2 ms



Total time elapsed: 00:00:00


In [13]:
E.head()

,_id,ltable_ID,rtable_ID,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,name_name_nmw,...,birth_year_birth_year_lev_sim,hourly_wage_hourly_wage_exm,hourly_wage_hourly_wage_anm,hourly_wage_hourly_wage_lev_dist,hourly_wage_hourly_wage_lev_sim,zipcode_zipcode_exm,zipcode_zipcode_anm,zipcode_zipcode_lev_dist,zipcode_zipcode_lev_sim,label
0,0,a2,b3,0.500000,0.5,0.333333,0.823932,4,0.750000,9.0,...,1.00,1,1.000000,0.0,1.00,1,1.000000,0.0,1.0,1
1,1,a3,b2,0.380952,0.5,0.333333,0.807792,4,0.714286,7.0,...,1.00,1,1.000000,0.0,1.00,1,1.000000,0.0,1.0,1
2,2,a5,b5,0.454545,0.5,0.333333,0.859487,3,0.800000,10.0,...,1.00,1,1.000000,0.0,1.00,1,1.000000,0.0,1.0,1
3,3,a1,b1,0.000000,0.0,0.000000,0.393939,11,0.000000,0.0,...,0.75,0,0.983333,3.0,0.25,1,1.000000,0.0,1.0,0
4,4,a2,b2,0.000000,0.0,0.000000,0.501263,12,0.250000,-1.0,...,0.75,0,0.859375,3.0,0.25,0,0.999841,2.0,0.6,0


time: 65 ms


In [15]:
from py_entitymatching.matcherselector.mlmatcherselection import get_xy_data

time: 1.21 ms


In [19]:
x,y=get_xy_data(x=None, y=None, table=E, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')

time: 3.77 ms


In [22]:
from sklearn.cross_validation import KFold, cross_val_score

time: 721 µs


In [78]:
k=9
random_state=None
cv = KFold(len(y), k, shuffle=True, random_state=random_state)

time: 1.7 ms


In [79]:
for t, s in cv:
    print("%s, %s" %(t, s))

[0 1 3 4 5 6 7 8], [2 9]
[0 1 2 3 5 6 7 8 9], [4]
[0 1 2 3 4 6 7 8 9], [5]
[0 1 2 3 4 5 6 8 9], [7]
[0 2 3 4 5 6 7 8 9], [1]
[0 1 2 3 4 5 6 7 9], [8]
[0 1 2 3 4 5 7 8 9], [6]
[1 2 3 4 5 6 7 8 9], [0]
[0 1 2 4 5 6 7 8 9], [3]
time: 7.81 ms


In [27]:
from sklearn import cross_validation

time: 819 µs


In [29]:
rf = em.RFMatcher()

time: 829 µs


In [73]:
predicted = cross_validation.cross_val_predict(rf.clf, x, y, cv=5)

time: 60.2 ms


/Users/pradap/miniconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


In [ ]:
for t, s in 

In [44]:
from sklearn.ensemble import RandomForestClassifier

time: 751 µs


In [53]:
clf = RandomForestClassifier(n_estimators=11, max_depth=None,
    min_samples_split=1, random_state=0)

time: 965 µs


In [54]:
for train_index, test_index in cv:
    tr_x = x[train_index]
    tr_y = y[train_index]
    ts_x = x[test_index]
    ts_y = y[test_index]

time: 1.95 ms


In [55]:
ts_y, tr_y

(array([1, 1, 0, 0, 0]), array([1, 0, 0, 0, 0]))

time: 3.61 ms


In [57]:
classifier = clf.fit(tr_x, tr_y)
predictions = classifier.predict_proba(ts_x)
pr_y = classifier.predict(ts_x)
print(predictions)
print(pr_y)

[[ 0.45454545  0.54545455]
 [ 0.45454545  0.54545455]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]]
[1 1 0 0 0]
time: 16.5 ms


In [58]:
import pandas as pd

time: 740 µs


In [66]:
a1 = L.loc[test_index]

time: 1.77 ms


In [65]:
b1 = pd.DataFrame({'predictions':pr_y, 'prob_false':predictions[:,0], 'prob_true':predictions[:,1]})

time: 1.77 ms


In [69]:
b1.index = a1.index

time: 755 µs


In [71]:
pd.concat([a1, b1], axis=1)

,_id,ltable_ID,rtable_ID,label,predictions,prob_false,prob_true
0,0,a2,b3,1,1,0.454545,0.545455
1,1,a3,b2,1,1,0.454545,0.545455
3,3,a1,b1,0,0,1.000000,0.000000
4,4,a2,b2,0,0,1.000000,0.000000
7,7,a3,b1,0,0,1.000000,0.000000


time: 10.9 ms


In [80]:
train_index, test_index

(array([2, 5, 6, 8, 9]), array([0, 1, 3, 4, 7]))

time: 3.91 ms


In [91]:
feats = list_diff(list(E.columns), ['_id', 'ltable_ID', 'rtable_ID', 'label'])

time: 966 µs


In [97]:
E1 = E.loc[train_index][feats]
E2 = E.loc[train_index]['label']
G1 = E.loc[test_index][feats]
G2 = E.loc[test_index]['label']

time: 7.28 ms


In [95]:
classifier = clf.fit(E1, E2)

time: 14.1 ms


In [98]:
predictions = classifier.predict_proba(G1)
# pr_y = classifier.predict(ts_x)
# print(predictions)
# print(pr_y)

time: 3.39 ms


In [100]:
pd.DataFrame(predictions)

,0,1
0,0.454545,0.545455
1,0.454545,0.545455
2,1.000000,0.000000
3,1.000000,0.000000
4,1.000000,0.000000


time: 7.51 ms


In [ ]:
pr_y = classifier.predict(G1)